In [12]:
import re
import glob
import os
import pandas as pd
import numpy as np
from io import StringIO 
import requests
import xml.etree.ElementTree as ET
import shutil, pickle

In [13]:
# This script prepares the dataframe with structural homolgy features. taken from FoldSeek run and from additional calculations (contact order).
# These features, of similarity between homologs, will be used in the next script (filter_process) to filter the homologs.


### a function to go over all files in folder

def walklevel(some_dir, level=0):
    some_dir = some_dir.rstrip(os.path.sep)
    assert os.path.isdir(some_dir)
    num_sep = some_dir.count(os.path.sep)
    for root, dirs, files in os.walk(some_dir):
        yield root, dirs, files
        num_sep_this = root.count(os.path.sep)
        if num_sep + level <= num_sep_this:
            del dirs[:]


def max_aln_no_gap(seq, max_gap_p = 0.05):
    start = 0
    gap_length = 0
    pos = 0
    max_length = 0 
    max_start = 0
    count = 0

    max_gap = max_gap_p * len(seq)
    
    while(pos < len(seq)):
        res = seq[pos]
        
        if (res == "-"):
            gap_length = gap_length + 1
        else:
            gap_length = 0
            
        if(max_gap < gap_length):
            if pos - start > max_length:
                max_length = pos - start - gap_length
                max_start = start
                
#             print(seq[pos])
            while (pos + 1 < len(seq)) & (seq[pos + 1] == "-"):
                pos = pos + 1
#                 print(pos)
            
            gap_length = 0
            start = pos + 1
            
        pos = pos + 1
        
        if (pos == len(seq)) & (pos - start > max_length):
            max_length = pos - start - gap_length
            max_start = start
    
    if max_length == 0:
        max_length = len(seq) 

        
    return max_start, max_start + max_length - 1, max_length

def max_seq_no_gap(seq, max_gap = 0.05):
    start, end, length = max_aln_no_gap(seq, max_gap)
    return seq[start : end]

def all_seq_ungap_index(seq, max_gap = 0.05):
    start, end, length = max_aln_no_gap(seq, max_gap)
    count = 0
    ungap_start = 0
    ungap_end = 0
#     print(len(seq))
    for i,c in enumerate(seq):
        if c == "-":
            count = count + 1
        if i == start:
            ungap_start = start - count
        if i == end - 1:
            
            ungap_end = end - count
            
    return seq.replace("-", "") ,ungap_start, ungap_end
            
        

### Foldseek tables calc

First, we run foldseek to calculate structure similarity between all virus pdbs to all human pdbs.
We run it twice. First when the human is the query and the virus is the target and secondly, the other way. The reason for doing this, is because requirements of filtering foldseek results (some of the results differ in numbers when running in each way). 
The below script should be embedded in a bash file, to be run in the cluster, and we give it as a generic example. 

In [3]:
### save it as foldseek_run.sh in a folder with the name of the virus in prefix exp_.
### for example here exp_adenoviruse55
### in the folder you need to have folder of pdbs called - {virus_name}_pdb (example - adenoviruse55_pdb)
### and a folder of human pdbs - human_pdb
### run this script as bash command after you foldseek in your machine
### the output is two tsv file, one for each direction - virus to human and human to virus 
### output name: foldseek_results_human_v2h_${virus_name}.tsv, foldseek_results_human_h2v_${virus_name}.tsv


var=$(pwd)
mydir="$(basename $var)"
echo $mydir
virus_name="${mydir#*_}"

module load foldseek

mkdir -p ${virus_name}_db

foldseek createdb ${virus_name}_pdb/ ${virus_name}_db/${virus_name}
foldseek createdb human_pdb/ human_db/human

mkdir -p foldseek_results_human_v2h_${virus_name}

mkdir -p foldseek_results_human_h2v_${virus_name}

foldseek search ${virus_name}_db/${virus_name} ../Human_db_folder/Human_db foldseek_results_human_v2h_${virus_name}/aln foldseek_results_human_v2h_${virus_name}/tmp -a --alignment-type 1
foldseek convertalis  ${virus_name}_db/${virus_name} ../Human_db_folder/Human_db foldseek_results_human_v2h_${virus_name}/aln foldseek_results_human_v2h_${virus_name}.tsv  --format-mode 4 --format-output "query,target,fident,alnlen,cigar,gapopen,qstart,qend,tstart,tend,qlen,tlen,evalue,bits,qaln,taln,qcov,tcov,qca,tca,alntmscore"

foldseek search ../Human_db_folder/Human_db ${virus_name}_db/${virus_name}  foldseek_results_human_h2v_${virus_name}/aln foldseek_results_human_h2v_${virus_name}/tmp -a --alignment-type 1
foldseek convertalis  ../Human_db_folder/Human_db ${virus_name}_db/${virus_name} foldseek_results_human_h2v_${virus_name}/aln foldseek_results_human_h2v_${virus_name}.tsv  --format-mode 4 --format-output "query,target,fident,alnlen,cigar,gapopen,qstart,qend,tstart,tend,qlen,tlen,evalue,bits,qaln,taln,qcov,tcov,qca,tca,alntmscore"


SyntaxError: invalid syntax (2619591828.py, line 13)

### extracting pLDDT scores from AlphaFold outputs 

This is a python code to get alphafold confidence results.
For each protein, we get the plddt scores per position for each model run (5 models).
We return a pandas dataframe with three columns: 
protein_name - uniprot,
score array - array of all the plddt results for this protein and this model.
model - the number of the model (0-4)
We run this script for each virus, such that we will obtain all viruses' protein pLDDT resluts per model run.

In [10]:
virus_name = "vaccinia_wr"

In [11]:
dst =  virus_name + "/plddt.csv"
src = "viruses_sequences/" + virus_name + "_pdb/"


print(src)
protein_array = []
model_array = []
result_array = []
dirs =  next(os.walk(src))[1]
for index, dir1 in enumerate(dirs):
	print(dir1)
	files = next(os.walk(src + dir1))[2]
	for file in files:
		if "result" in file:
			print(file)
			protein_array.append(dir1)
			model_array.append(file.split(".")[0])
			with open(src + dir1 + "/" + file, 'rb') as f:
				print(file)
				try:
					data = pickle.load(f)
				except Exception as e:
					print(e)


				result_array.append(data["plddt"])



result_data = {'protein': protein_array, 'model': model_array, 'score_array':result_array}
result_pd = pd.DataFrame(data=result_data)
result_pd.to_csv(dst, sep='\t')

viruses_sequences/vaccinia_wr_pdb/


StopIteration: 

### Clean schema from the two-way running of FoldSeek

This script takes the two foldseek tables we have for each virus - 
foldseek_results_human_v2h_{virus_name}, foldseek_results_human_h2v_{virus_name}
(since we run it for virus proteins as a query and human protens as target and vice versa) and combines it all to one table with all viral protein results with one schema (query = virus protein)

In [17]:
# make all the columns the same in both dataframes such that 
# query and target are changed to virus protein and human protein, respectively

def read_virus_both(virus, db="human", path = "foldseek_table/"):
    fav_cols = ['query', 'target','alnlen', 'cigar', 'qstart',
       'qend', 'tstart', 'tend', 'qlen', 'tlen', 'bits', 'qaln', 'taln', 'alntmscore']
    q_t_columns = ['start', 'end', 'len', 'aln']
    df_v2h = pd.DataFrame()
    df_h2v = pd.DataFrame()
    try:
        df_v2h =  pd.read_csv(path+ "foldseek_results_{}_v2h_{}.tsv".format(db, virus), sep = '\t')[fav_cols]
    except:
        print("not found v2h")

    try:
        df_h2v = pd.read_csv(path+ "foldseek_results_{}_h2v_{}.tsv".format(db, virus), sep = '\t')[fav_cols]
    except:
        print("not found h2v")


    if ((df_h2v.empty) & (df_v2h.empty)):
        return pd.DataFrame()

    elif ((df_h2v.empty) & (df_v2h.empty == False)):
        df_v2h["virus_protein"] = df_v2h["query"].apply(lambda x: x.split(".")[0])
        df_v2h["human_protein"] = df_v2h["target"].apply(lambda x: x.split("-")[1] + "-" + x.split("-")[2])
        df_v2h.drop(["query", "target"], axis = 1, inplace = True)
        df_v2h["side"] = 1
        both_sides = df_v2h.copy()


    elif ((df_h2v.empty == False) & (df_v2h.empty)):
        for col in q_t_columns:
            df_h2v["temp" + col] = df_h2v["q" + col]
            df_h2v["q" + col] = df_h2v["t" + col]
            df_h2v["t" + col] = df_h2v["temp" + col]
            df_h2v.drop("temp" + col, inplace = True, axis = 1)
        df_h2v["virus_protein"] = df_h2v["target"].apply(lambda x: x.split(".")[0])
        df_h2v["human_protein"] = df_h2v["query"].apply(lambda x: x.split("-")[1] + "-" + x.split("-")[2])
        df_h2v.drop(["query", "target"], axis = 1, inplace = True)
        df_h2v["side"] = -1
        both_sides = df_h2v.copy()
    else:

        for col in q_t_columns:
            df_h2v["temp" + col] = df_h2v["q" + col]
            df_h2v["q" + col] = df_h2v["t" + col]
            df_h2v["t" + col] = df_h2v["temp" + col]
            df_h2v.drop("temp" + col, inplace = True, axis = 1)

        df_v2h["virus_protein"] = df_v2h["query"].apply(lambda x: x.split(".")[0])
        df_v2h["human_protein"] = df_v2h["target"].apply(lambda x: x.split("-")[1] + "-" + x.split("-")[2])
        df_h2v["virus_protein"] = df_h2v["target"].apply(lambda x: x.split(".")[0])
        df_h2v["human_protein"] = df_h2v["query"].apply(lambda x: x.split("-")[1] + "-" + x.split("-")[2])
        df_v2h.drop(["query", "target"], axis = 1, inplace = True)
        df_h2v.drop(["query", "target"], axis = 1, inplace = True)
        
        df_v2h["side"] = 1
        df_h2v["side"] = -1

       
        both_sides = pd.concat([df_v2h, df_h2v])


    # calculate how many matches there are for each viral protein from both sides of the FoldSeek run (v2h and h2v)
    
    sides_sum_df = both_sides.groupby(["virus_protein", "human_protein"]).agg({"side":"sum"}).reset_index()
    sides_sum_df.columns = ["virus_protein", "human_protein", "sum_sides"]
    
    
    both = pd.merge(both_sides,sides_sum_df, on=["virus_protein", "human_protein"])
    both["virus"] = virus
    
    return both






path = "data/"
virus_array = []


# run over all the virus results (in this example - only vaccinia)

for root, dirs, files in walklevel(path):
    if dirs == []:
        break
        # print(files)
    for dir1 in dirs[:]:
        print(dir1)
        if dir1.split("_")[0] == "exp":
            try:
                virus_name = dir1.replace("exp_", "")
                human_virus_df = read_virus_both(virus_name, db="human", path=path + "/" + dir1 + "/")
                human_virus_df["innate"] = 0
                virus_array.append(human_virus_df)



            except:
                print("error")


matches_df = pd.concat(virus_array)
### we do not use this feature in the final analysis, where we idetify whether these human proteins belong to the innate immune system
matches_df = matches_df[matches_df["innate"] == 0]

display(matches_df)


.ipynb_checkpoints
exp_vaccinia_wr


,alnlen,cigar,qstart,qend,tstart,tend,qlen,tlen,bits,qaln,taln,alntmscore,virus_protein,human_protein,side,sum_sides,virus,innate
0,289,3M47D6M1I24M1I19M1I4M1D28M1D1I2M1D1M2D45M12D2M...,2,166,317,601,166,623,83,DSG-------------------------------------------...,GEGLGTVCTGVVMENNTIIVAGEASASKLSRQKNKNVEIYRYHDRG...,0.8614,Q76ZL3,Q3ZCT8-F1,1,1,vaccinia_wr,0
1,174,32M1D1M2D16M1I7M1D26M6I45M4D8M2D1M1D20M,4,166,311,477,166,571,83,GIYETPINYKKSNVSAVSVNNTIFVTGGLFIN-N--SNSTIVVNNM...,WIGLAPLNIPRYEFGICVLDQKVYVIGGIATNVRPGVTIRKHENSV...,0.8569,Q76ZL3,Q9NXS3-F1,1,1,vaccinia_wr,0
2,176,32M4I16M1I4M1D29M2D1M1D47M6D8M2D1M1D20M,4,166,320,490,166,589,82,GIYETPINYKKSNVSAVSVNNTIFVTGGLFINNSNSTIVVNNMEKL...,IIPKADLPSPRKEFSASAIGCKVYVTGGRGSE----NGVSKDVWVY...,0.8508,Q76ZL3,Q9H0H3-F1,1,1,vaccinia_wr,0
3,185,34M7D16M1I6M1D26M5I38M1D8M10D8M2D1M1D20M,4,166,346,524,166,644,82,GIYETPINYKKSNVSAVSVNNTIFVTGGLFINNS-------NSTIV...,WRSLTQLPTPLLGHSVCTAGNFLFVLGGESPSGSASSPLADDSRVV...,0.8503,Q76ZL3,Q8N239-F1,1,1,vaccinia_wr,0
4,174,34M1I14M1I7M1D26M5I46M6D8M2D1M1D8M1D6M1I5M,4,166,312,477,166,581,82,GIYETPINYKKSNVSAVSVNNTIFVTGGLFINNSNSTIVVNNMEKL...,WQSLAKLPTRLYKASAITLHRSIYVLGGMAVSSG-RSLVSHNVYIF...,0.8485,Q76ZL3,Q2WGJ6-F1,1,1,vaccinia_wr,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15562,74,74M,34,107,34,107,110,110,50,AIVKADEDDNEETLKQRLTNLEKKITNVTTKFEQIEKCCKRNDEVL...,ALFTYPKGAGEMLEDGSERFLCESVFSYQVASTLKQVKHDQQVARM...,0.7394,P11258,Q96DE5-F1,-1,0,vaccinia_wr,0
15563,343,1M2D1I5M1I2M2I7M2I2M2I1M3I1M1I19M2D4M1D47M2D22...,543,844,1,260,844,264,67,NND-KFRLN-PE--VSYFTNK--RT--R---G-PLGILSNYVKTLL...,M--ESGFTSKDTYLSHFNPRDYLEKYYKFGSRHSAESQILKHLLKN...,0.6991,P04298,P40261-F1,-1,-1,vaccinia_wr,0
15564,150,33M1I34M1I79M1D1M,482,629,69,217,725,217,51,GSDLLEIDKKTIRELRESLDREREMRSELEKEL-DTIRNGKVDGSC...,MYSNRMRSYKQEMGKLETDFKRSRIAYSDEVRNELLGDDGNSSENQ...,0.7396,P24759,Q96AJ9-F1,-1,-1,vaccinia_wr,0
15565,281,6M1D1M30D19I26M2D2M19D2M23D5M1I8M2I1M7I21M3I4M...,7,234,23,222,248,227,50,SIISQIIKYNRRLAKSIICEDDSQIITLTAFVNQCLWC--------...,LIDEND-N------------------------------KIGAETKK...,0.5283,P04312,Q13907-F1,-1,-1,vaccinia_wr,0


this script adds columns kater used for filtring and adding for each protein its contac order value

In [18]:
import re
import glob
import os
import pandas as pd
import numpy as np
from io import StringIO 
import requests
import xml.etree.ElementTree as ET



# get data frame with virus proteins, human protein it matches, if it is innate or not, side of foldseek direction (v2h 1 or h2v -1) 
# and csu result for each position 
# the function returns for each vira protein its abs mean contact order value (a measure for fold complexity) - sum_residual_dist / count_residual_dist
# it is optional to calculate contact order only between positions - using pos_array and to filter contact only inside those position with - residu_filter 
def calc_contact_order(df, pos_array=["qstart", "qend"], residu_filter = True, seq_distance = 4, prefix=""):
    df = df[df["residual_dist"] >= seq_distance] # we compute contact ordre only for residues which are at least 4AA apart from each other
    if pos_array != []:
        df = df[df["amino_acid_number"] >= df[pos_array[0]] + 1]
        df = df[df["amino_acid_number"] <= df[pos_array[1]] + 1]

        if residu_filter == True:
            df = df[df["contact_acid_number"] >= df[pos_array[0]] + 1]
            df = df[df["contact_acid_number"] <= df[pos_array[1]] + 1]


    amino_dist = (df.groupby(["virus", "virus_protein", "human_protein", "side"])
                             .agg(sum_residual_dist=("residual_dist","sum"),
                                  count_residual_dist=("residual_dist","count"))
                             .reset_index()
                            )
    
    amino_dist.columns = ["virus", "virus_protein", "human_protein", "side", "sum_residual_dist", "count_residual_dist"]
    amino_dist[prefix + "abs_mean_contact_order"] = amino_dist["sum_residual_dist"] / amino_dist["count_residual_dist"]
    amino_dist.drop(["sum_residual_dist", "count_residual_dist"], axis=1, inplace=True)
    return amino_dist



    
def total_median(array):
    if array == None:
        return 0
    return np.median(array)


def median_by_pos(start, end, array):
    if array == None:
        return 0
    pos_array = np.array(array)[start -1 : end]
    return np.median(pos_array)

# connect the csu table with the foldseek table.
def virus_csu_func(virus_csu_df, foldseek_filtered_virus, prefix="virus_"):
    virus_csu_df["virus_protein"] = virus_csu_df["target_protein"]
    virus_csu_df.drop("target_protein", axis =1, inplace = True)

    # calculate contact order only between the matched regions (only position between start and end matching positions)
    virus_csu_with_index = pd.merge(virus_csu_df, foldseek_filtered_virus, on = ["virus", "virus_protein"])
    virus_contact_order_df = calc_contact_order(virus_csu_with_index, prefix = prefix)
    virus_pivot_with_mean = foldseek_filtered_virus.merge(virus_contact_order_df, on=["virus", "virus_protein", "human_protein", "side"])

    #  calc contact order over the full protein sequence 
    full_prefix = "full_" + prefix
    full_virus_contact_order_df = calc_contact_order(virus_csu_with_index, pos_array=[], prefix = full_prefix)
    full_virus_pivot_with_mean = virus_pivot_with_mean.merge(full_virus_contact_order_df, on=["virus", "virus_protein", "human_protein", "side"])

    return full_virus_pivot_with_mean


matches_df["qungap"] = matches_df["qaln"].apply(lambda x: max_seq_no_gap(x))
matches_df["tungap"] = matches_df["taln"].apply(lambda x: max_seq_no_gap(x))
matches_df["qungap_len"] = matches_df["qungap"].apply(lambda x: len(x))
matches_df["tungap_len"] = matches_df["tungap"].apply(lambda x: len(x))
matches_df["ratio_ungap"] = matches_df["qungap_len"] / matches_df["tungap_len"]
    
matches_df["ratio"] = matches_df["qlen"] / matches_df["tlen"]

### read csu table with contact information betwen residues (here we read only vaccinia as an example)
csu_df = pd.read_csv(f"data/exp_vaccinia_wr/csu_tabel_{virus_name}.csv")

virus_csu = virus_csu_func(csu_df, matches_df[['virus', 'virus_protein', 'human_protein', "innate", 'side', 'qstart', 'qend']], prefix="virus_")
virus_csu.drop(['qstart', 'qend'], axis = 1, inplace = True)
virus_csu_df = pd.merge(virus_csu, matches_df, on = ['virus', 'virus_protein', 'human_protein', "innate", 'side'], how="right").fillna(0)
display(virus_csu_df)



,virus,virus_protein,human_protein,innate,side,virus_abs_mean_contact_order,full_virus_abs_mean_contact_order,alnlen,cigar,qstart,...,qaln,taln,alntmscore,sum_sides,qungap,tungap,qungap_len,tungap_len,ratio_ungap,ratio
0,vaccinia_wr,Q76ZL3,Q3ZCT8-F1,0,1,26.806833,26.806833,289,3M47D6M1I24M1I19M1I4M1D28M1D1I2M1D1M2D45M12D2M...,2,...,DSG-------------------------------------------...,GEGLGTVCTGVVMENNTIIVAGEASASKLSRQKNKNVEIYRYHDRG...,0.8614,1,IYETPINYKKSNVSAVSVNNTIFVTGGLFINNSNSTIVVNNMEKLD...,GEGLGTVCTGVVMENNTIIVAGEASASKLSRQKNKNVEIYRYHDRG...,185,288,0.642361,0.266453
1,vaccinia_wr,Q76ZL3,Q9NXS3-F1,0,1,26.806833,26.806833,174,32M1D1M2D16M1I7M1D26M6I45M4D8M2D1M1D20M,4,...,GIYETPINYKKSNVSAVSVNNTIFVTGGLFIN-N--SNSTIVVNNM...,WIGLAPLNIPRYEFGICVLDQKVYVIGGIATNVRPGVTIRKHENSV...,0.8569,1,GIYETPINYKKSNVSAVSVNNTIFVTGGLFIN-N--SNSTIVVNNM...,WIGLAPLNIPRYEFGICVLDQKVYVIGGIATNVRPGVTIRKHENSV...,173,173,1.000000,0.290718
2,vaccinia_wr,Q76ZL3,Q9H0H3-F1,0,1,26.806833,26.806833,176,32M4I16M1I4M1D29M2D1M1D47M6D8M2D1M1D20M,4,...,GIYETPINYKKSNVSAVSVNNTIFVTGGLFINNSNSTIVVNNMEKL...,IIPKADLPSPRKEFSASAIGCKVYVTGGRGSE----NGVSKDVWVY...,0.8508,1,GIYETPINYKKSNVSAVSVNNTIFVTGGLFINNSNSTIVVNNMEKL...,IIPKADLPSPRKEFSASAIGCKVYVTGGRGSE----NGVSKDVWVY...,175,175,1.000000,0.281834
3,vaccinia_wr,Q76ZL3,Q8N239-F1,0,1,26.806833,26.806833,185,34M7D16M1I6M1D26M5I38M1D8M10D8M2D1M1D20M,4,...,GIYETPINYKKSNVSAVSVNNTIFVTGGLFINNS-------NSTIV...,WRSLTQLPTPLLGHSVCTAGNFLFVLGGESPSGSASSPLADDSRVV...,0.8503,1,GIYETPINYKKSNVSAVSVNNTIFVTGGLFINNS-------NSTIV...,WRSLTQLPTPLLGHSVCTAGNFLFVLGGESPSGSASSPLADDSRVV...,141,184,0.766304,0.257764
4,vaccinia_wr,Q76ZL3,Q2WGJ6-F1,0,1,26.806833,26.806833,174,34M1I14M1I7M1D26M5I46M6D8M2D1M1D8M1D6M1I5M,4,...,GIYETPINYKKSNVSAVSVNNTIFVTGGLFINNSNSTIVVNNMEKL...,WQSLAKLPTRLYKASAITLHRSIYVLGGMAVSSG-RSLVSHNVYIF...,0.8485,1,GIYETPINYKKSNVSAVSVNNTIFVTGGLFINNSNSTIVVNNMEKL...,WQSLAKLPTRLYKASAITLHRSIYVLGGMAVSSG-RSLVSHNVYIF...,173,173,1.000000,0.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15562,vaccinia_wr,P11258,Q96DE5-F1,0,-1,4.188976,4.237179,74,74M,34,...,AIVKADEDDNEETLKQRLTNLEKKITNVTTKFEQIEKCCKRNDEVL...,ALFTYPKGAGEMLEDGSERFLCESVFSYQVASTLKQVKHDQQVARM...,0.7394,0,AIVKADEDDNEETLKQRLTNLEKKITNVTTKFEQIEKCCKRNDEVL...,ALFTYPKGAGEMLEDGSERFLCESVFSYQVASTLKQVKHDQQVARM...,73,73,1.000000,1.000000
15563,vaccinia_wr,P04298,P40261-F1,0,-1,57.930200,72.836754,343,1M2D1I5M1I2M2I7M2I2M2I1M3I1M1I19M2D4M1D47M2D22...,543,...,NND-KFRLN-PE--VSYFTNK--RT--R---G-PLGILSNYVKTLL...,M--ESGFTSKDTYLSHFNPRDYLEKYYKFGSRHSAESQILKHLLKN...,0.6991,-1,GIK--TK--YYKFDYIQETIRSDTFVSSVREVFYFGKFNIIDWQFA...,M--ESGFTSKDTYLSHFNPRDYLEKYYKFGSRHSAESQILKHLLKN...,212,212,1.000000,3.196970
15564,vaccinia_wr,P24759,Q96AJ9-F1,0,-1,8.444444,37.194256,150,33M1I34M1I79M1D1M,482,...,GSDLLEIDKKTIRELRESLDREREMRSELEKEL-DTIRNGKVDGSC...,MYSNRMRSYKQEMGKLETDFKRSRIAYSDEVRNELLGDDGNSSENQ...,0.7396,-1,GSDLLEIDKKTIRELRESLDREREMRSELEKEL-DTIRNGKVDGSC...,MYSNRMRSYKQEMGKLETDFKRSRIAYSDEVRNELLGDDGNSSENQ...,149,149,1.000000,3.341014
15565,vaccinia_wr,P04312,Q13907-F1,0,-1,64.047838,64.315526,281,6M1D1M30D19I26M2D2M19D2M23D5M1I8M2I1M7I21M3I4M...,7,...,SIISQIIKYNRRLAKSIICEDDSQIITLTAFVNQCLWC--------...,LIDEND-N------------------------------KIGAETKK...,0.5283,-1,HKRVSVSAILLTTDNKILVCNRRDSFLYSEIIRTRNMFRKKRLFLN...,GCFTNTCCSHPLSNPAELEESDALGVRRAAQRRLKAELGIPLEEVP...,223,149,1.496644,1.092511


In [19]:
###### compute human protein contact order

human_csu_df = pd.read_csv("data/all_csu_table.tsv", sep = '\t')
human_csu_df["human_protein"] = human_csu_df["target_protein"]
human_csu_df.drop(["target_protein", "virus"], axis =1, inplace = True)

prefix = "human_"
human_csu_with_index = pd.merge(human_csu_df, virus_csu_df[['virus', 'virus_protein', 'human_protein', 'side', "tstart", "tend"]], on = ["human_protein"], how= "left").fillna(0)
human_contact_order_df = calc_contact_order(human_csu_with_index, pos_array=["tstart", "tend"], prefix = prefix)
human_mean_contact_order = pd.merge(human_contact_order_df, virus_csu_df[['virus', 'virus_protein', 'human_protein', 'side']], on = ['virus', "virus_protein", "human_protein", "side"])


prefix = "full_human_"
human_csu_with_index = pd.merge(human_csu_df, virus_csu_df[['virus', 'virus_protein', 'human_protein', 'side']].drop_duplicates(), on = ["human_protein"])
full_human_contact_order_df = calc_contact_order(human_csu_with_index, pos_array=[], prefix = prefix)
full_human_pivot_quantile_with_mean = pd.merge(full_human_contact_order_df, human_mean_contact_order, on = ['virus', "virus_protein", "human_protein", "side"])


before_filter_no_plddt_df = (pd.merge(full_human_pivot_quantile_with_mean, virus_csu_df, 
                                      on = ['virus', 'virus_protein', 'human_protein', 'side'])
                             .drop(["innate"], axis = 1)
                            )

before_filter_no_plddt_df.to_csv("data/before_filter_no_plddt.csv")                           
display(before_filter_no_plddt_df)    

,virus,virus_protein,human_protein,side,full_human_abs_mean_contact_order,human_abs_mean_contact_order,virus_abs_mean_contact_order,full_virus_abs_mean_contact_order,alnlen,cigar,...,qaln,taln,alntmscore,sum_sides,qungap,tungap,qungap_len,tungap_len,ratio_ungap,ratio
0,vaccinia_wr,P01136,H3BU77-F1,-1,4.250000,4.205882,4.074074,16.190972,42,42M,...,TTTSYIPSPGIMLVLVGIIIITCCLLSVYRFTRRTKLPIQDM,PEGPRQHHPSEVTERQLANKRIQNMQHLKKEKRRLNKRFSRP,0.7793,-1,TTTSYIPSPGIMLVLVGIIIITCCLLSVYRFTRRTKLPIQD,PEGPRQHHPSEVTERQLANKRIQNMQHLKKEKRRLNKRFSR,41,41,1.000000,2.058824
1,vaccinia_wr,P01136,Q5JQD4-F1,-1,13.352113,13.352113,17.199248,16.190972,138,4M1D17M3I1M1I8M68D6I4M1I21M1D2M,...,MSMKYLMLLFAAMIIRSFADSG---N-AIETTSPEITNATTDIPAI...,SVCR-PWPAVAIALLALLVCLGALVDTCPIKPEAP-----------...,0.6074,-1,MSMKYLMLLFAAMIIRSFADSG---N-AIETTSPEITNATTDIPAI...,GEDESLEELSHYYASLCHYLNVVTRQWWEGAD-M,137,34,4.029412,2.000000
2,vaccinia_wr,P01136,Q86Y28-F1,-1,4.000000,4.000000,4.000000,16.190972,32,24M1D4M1D1I1M,...,LVLVGIIIITCCLLSVYRFTRRTKLPIQDM-V,MAAGAVFLALSAQLLQARLMKEES-PVVS-WW,0.7624,-1,LVLVGIIIITCCLLSVYRFTRRTKLPIQDM-,MAAGAVFLALSAQLLQARLMKEES-PVVS-W,31,31,1.000000,3.589744
3,vaccinia_wr,P01136,Q9NRQ5-F1,-1,4.235955,4.125000,4.074074,16.190972,52,4M1I3M1I32M2I2M1D2I1M2I1M,...,PNTT-TSY-IPSPGIMLVLVGIIIITCCLLSVYRFTRRTKL--PIQ...,MRQLKGKPKKETSKDKKERKQAMQEARQQITTVVLPTLAVVVLLI-...,0.6814,-1,PNTT-TSY-IPSPGIMLVLVGIIIITCCLLSVYRFTRRTKL--PIQ...,MRQLKGKPKKETSKDKKERKQAMQEARQQITTVVLPTLAVVVLLI-...,51,51,1.000000,2.372881
4,vaccinia_wr,P03296,P51808-F1,-1,32.555556,32.516807,45.777778,51.960448,184,8M2I5M1D9M1D3M8D2I5M1I18M1D11M8D3I5M1D6M2I9M1D...,...,KGLQTIKL--FNNEFDCIRNDIRELFKHVTDSDSIQL--PMEDN-S...,EVGFNAEEAHNIVKE-CVDGVLGGE-DYN--------HNNINQWTA...,0.5567,-1,KGLQTIKL--FNNEFDCIRNDIRELFKHVTDSDSIQL--PMEDN-S...,EVGFNAEEAHNIVKE-CVDGVLGGE-DYN--------HNNINQWTA...,183,110,1.663636,2.853448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15533,vaccinia_wr,Q89121,Q9HBY8-F1,-1,75.593297,39.517024,68.402527,67.977589,484,7M5D4M2D8M2D1M51D17M6I8M2I22M6D16M90D10M1I12M5...,...,PEYQWMSPHRLSDTVILGDCLYFNNIMSQLDLHQNWAPSVRLLNYF...,NGNINLG-----PSAN--PNAQPTDF--D-----------------...,0.5335,-1,PEYQWMSPHRLSDTVILGDCLYFNNIMSQLDLHQNWAPSVRLLNYF...,KLYFVLDYVNGGELFFHLQRERR-----FLEPRARFYAAEVASAIG...,483,236,2.046610,1.196185
15534,vaccinia_wr,Q89121,Q9HC98-F1,-1,42.312916,42.439546,68.110557,67.977589,486,4M3D1M1D3M1D3M1D4M2D2M51D16M6I9M2I21M7D12M1D2M...,...,MSPHRLSDTVILGDCLYFNNIMSQLDLHQNWAPSVRLLNYFKNFNK...,RHPN---T-LSF-RCS-LADF--QI---------------------...,0.5890,-1,MSPHRLSDTVILGDCLYFNNIMSQLDLHQNWAPSVRLLNYFKNFNK...,NELNIVLELADAGDLSQMIKYFK-----KQKRLIPERTVWKYFVQL...,485,244,1.987705,1.402556
15535,vaccinia_wr,Q89121,Q9P1W9-F1,-1,38.861635,38.861635,68.427991,67.977589,492,6M8D4M2D1I7M2D2M51D16M6I17M5I16M3D13M1D3M89D9M...,...,SPEYQWMSPHRLSDTVILGD-CLYFNNIMSQLDLHQNWAPSVRLLN...,TPTPPP--------GGKD--REAFEAEY--RL--------------...,0.6062,-1,SPEYQWMSPHRLSDTVILGD-CLYFNNIMSQLDLHQNWAPSVRLLN...,GFMLVLERPLPAQDLFDYITEKGP-----LGEGPSRCFFGQVVAAI...,491,239,2.054393,1.411576
15536,vaccinia_wr,Q89121,Q9UIK4-F1,-1,42.771125,36.393019,67.977589,67.977589,495,13M9D3M2D7M2D2M51D16M6I13M5I17M7D12M1D2M89D10M...,...,MGVANDSSPEYQWMSPHRLSDTVILGDCLYFNNIMSQLDLHQNWAP...,MFQASMRSPNMEP---------FKQ--QKVEDFY--DI--------...,0.5294,-1,MGVANDSSPEYQWMSPHRLSDTVILGDCLYFNNIMSQLDLHQNWAP...,TDVVLILELVSGGELFDFLAQK-E----SLSEEEATSFIKQILDGV...,408,237,1.721519,1.186486
